In [103]:
import os
import urllib3
import shutil
import zipfile
import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [16]:
http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/fra-eng.zip'

with (http.request('GET', url, preload_content=False) as r,
    open('data/fra-eng.zip', 'wb') as out_file):
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile('data/fra-eng.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [104]:
lines_raw = pd.read_csv('data/fra.txt', names=['src', 'tar', 'lic'], sep='\t')
lines_raw.sample(10)

,src,tar,lic
183077,I'll understand if you don't want to go there ...,Je comprendrai si vous ne voulez pas y aller a...,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
47135,Do you not like girls?,N'aimes-tu pas les filles ?,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
126722,I hear that Tom can speak French.,J'entends que Tom sait parler français.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
184128,The delivery of the goods was delayed due to t...,La livraison des marchandises a été retardée à...,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
156896,You don't seem to want to come with us.,Il ne semble pas que vous veuillez venir avec ...,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
58586,This is your community.,C'est votre communauté.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
151220,I'd be unhappy if that happened again.,Je serais malheureux si ça se reproduisait.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
43180,It's a monster storm.,Il s'agit d'une tempête géante.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
61717,He never stops to think.,Il ne cesse jamais de réfléchir.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
165119,The stable is right behind the farm house.,L'écurie est juste derrière la ferme.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [36]:
lines = (lines_raw
    .drop('lic', axis=1)
    .iloc[:60000]
    .assign(tar = lambda x: '\t ' + x['tar'] + ' \n')
)

In [44]:
src_vocab = set()
for line in lines['src']:
    for ch in line:
        src_vocab.add(ch)

tar_vocab = set()
for line in lines['tar']:
    for ch in line:
        tar_vocab.add(ch)

In [52]:
src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))

In [58]:
src_to_index = {word: i+1 for i, word in enumerate(src_vocab)}
tar_to_index = {word: i+1 for i, word in enumerate(tar_vocab)}

In [79]:
encoder_input = lines['src'].apply(lambda row: [src_to_index[x] for x in row]).tolist()
decoder_input = lines['tar'].apply(lambda row: [tar_to_index[x] for x in row]).tolist()
decoder_target = lines['tar'].str.replace('\t ', '').apply(lambda row: [tar_to_index[x] for x in row]).tolist()

In [101]:
max_src_len = lines['src'].apply(lambda x: len(x)).max()
max_tar_len = lines['tar'].apply(lambda x: len(x)).max()

In [172]:
encoder_input = pad_sequences(encoder_input, padding='post', maxlen=max_src_len)
decoder_input = pad_sequences(decoder_input, padding='post', maxlen=max_tar_len)
decoder_target = pad_sequences(decoder_target, padding='post', maxlen=max_tar_len)

endocer_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

MemoryError: Unable to allocate 3.57 GiB for an array with shape (478800000, 2) and data type float32

In [ ]:
endocer_input

In [138]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [169]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_23 (InputLayer)          [(None, None, 80)]   0           []                               
                                                                                                  
 input_24 (InputLayer)          [(None, None, 105)]  0           []                               
                                                                                                  
 lstm_17 (LSTM)                 [(None, 256),        345088      ['input_23[0][0]']               
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                            

In [166]:
encoder_input.shape, decoder_input.shape, max_src_len, max_tar_len, src_vocab_size, tar_vocab_size

((60000, 23), (60000, 76, 105), 23, 76, 80, 105)

In [170]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=40, validation_split=0.2)

Epoch 1/40


ValueError: in user code:

    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\input_spec.py", line 213, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer